<!-- dom:TITLE: Higher order sensitivity indices for non-additive models -->
# Higher order sensitivity indices for non-additive models
<!-- dom:AUTHOR: Leif Rune Hellevik at Department of Structural Engineering, NTNU -->
<!-- Author: --> **Leif Rune Hellevik**, Department of Structural Engineering, NTNU

<!-- dom:AUTHOR: Vinzenz Gregor Eck at Expert Analytics -->
<!-- Author: --> **Vinzenz Gregor Eck**, Expert Analytics

Date: **Mar 21, 2017**

In [1]:
%matplotlib inline

%matplotlib inline
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import chaospy as cp
from sensitivity_examples_nonlinear import calculate_sensitivity_indices_non_additive_model
import monte_carlo

# Introduction
<div id="sec:introduction"></div>

This chapter introduces to higher order sensitivity indices.
These indices are needed if for sensitivity analysis of non-additive models.

In numerical analysis we usually differ between linear and non-linear models.A linear model is for example:

<!-- Equation labels as ordinary links -->
<div id="eq:linear_model"></div>

$$
\begin{equation}
Y =  \sum_{i=1}^{r} \Omega_i \, Z_i
\label{eq:linear_model} \tag{1}
\end{equation}
$$

A non-linear variation of this model:

<!-- Equation labels as ordinary links -->
<div id="eq:nonlinear_model"></div>

$$
\begin{equation}
Y = \sum_{i=1}^{r} \Omega_i \, Z_i^2
\label{eq:nonlinear_model} \tag{2}
\end{equation}
$$

For sensitivity analysis we introduce another model property: additive and non-additive.

Definition: A model is additive if it is possible to separate the effect of its input variables in a
variance decompositionframework.

That means for an additive model:

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\sum_{i=1}^{r} S_i = 1,
\label{_auto1} \tag{3}
\end{equation}
$$

whereas for an non-additive model:

<!-- Equation labels as ordinary links -->
<div id="_auto2"></div>

$$
\begin{equation}
\sum_{i=1}^{r} S_i < = 1.
\label{_auto2} \tag{4}
\end{equation}
$$

Both models ([eq:linear_model](#eq:linear_model)) and ([eq:nonlinear_model](#eq:nonlinear_model)) are additive models.
An example for an non-additive model is:

<!-- Equation labels as ordinary links -->
<div id="_auto3"></div>

$$
\begin{equation}
Y = \prod_{i=1}^{r} \Omega_i \, Z_i^2,
\label{_auto3} \tag{5}
\end{equation}
$$

since all input variables are multiplied with each other, it is not possible to separate the effects with
variance decomposition.
For this reason another measure is needed it the model is non-additive.
In the previous chapter we also discussed the standardized regression coefficients $\beta_i$ as sensitivity measure.
However, these measures have an even stronger restriction since:

<!-- Equation labels as ordinary links -->
<div id="eq:reg_sum"></div>

$$
\begin{equation}
\sum_{i=1}^{r} \beta_i^2 = 1 \label{eq:reg_sum} \tag{6}
\end{equation}
$$

is only valid for linear (thus, additive) models.

Before we introduce higher order sensitivity coefficients we will repeat our analysis of the first example for a non-additive model.

# The non-additive model
<div id="sec:non_additive_model"></div>

Now we consider the same numerical model as before:

<!-- Equation labels as ordinary links -->
<div id="eq:non_add_model"></div>

$$
\begin{equation}
Y = \sum_{i=1}^{r} \Omega_i \, Z_i
\label{eq:non_add_model} \tag{7}
\end{equation}
$$

but we now we consider $\Omega_i$ as well as random input factors.

We define the input factors as normal distributed random variables:

<!-- Equation labels as ordinary links -->
<div id="eq:NZi"></div>

$$
\begin{equation}
Z_i \sim N(0, \sigma_{Z_i}), \qquad i=1,2, \ldots, r \label{eq:NZi} \tag{8} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:NOi"></div>

$$
\begin{equation} 
\Omega_i \sim N(\mu_{\Omega_i}, \sigma_{\Omega_i}), \qquad i=1,2, \ldots, r. \label{eq:NOi} \tag{9} 
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="_auto4"></div>

$$
\begin{equation} 
\label{_auto4} \tag{10}
\end{equation}
$$

The distributions of $Z_i$ are not changed.
The mean values $\mu_{\Omega_i}$ of the distributions of $\Omega_i$ are chosen to be non-zero:

<!-- Equation labels as ordinary links -->
<div id="_auto5"></div>

$$
\begin{equation}
\mu_{\Omega_i} = 0.5 i, \qquad i=1,2, \ldots, r.
\label{_auto5} \tag{11}
\end{equation}
$$

The python code for the model is:

In [2]:
# start the linear model
def linear_model(w, z):
    assert w.shape == z.shape
    return np.sum(w*z, axis=1)

# Definition of the random input variables

Again we use the chaospy package to define our random input variables in a function.
In this example we choose to set $r=4$.

The definition of all marginal and the joint distribution is:

In [3]:
# definition of the distributions for r = 4
def generate_distributions():

    # define marginal distributions
    Z_1 = cp.Normal(0, 1)
    Z_2 = cp.Normal(0, 2)
    Z_3 = cp.Normal(0, 3)
    Z_4 = cp.Normal(0, 4)

    O_1 = cp.Normal(0.5*1, 1)
    O_2 = cp.Normal(0.5*2, 2)
    O_3 = cp.Normal(0.5*3, 3)
    O_4 = cp.Normal(0.5*4, 4)

    # define joint distributions
    joint_distribution = cp.J(Z_1, Z_2, Z_3, Z_4, O_1, O_2, O_3, O_4)
    return joint_distribution

# First order sensitivity coefficients

In [4]:
    # sensitivity reference values
    S_book = np.array([0.0006, 0.009, 0.046, 0.145, 0, 0, 0, 0])
    St_book = np.array([0.003, 0.045, 0.229, 0.723, 0.002, 0.036, 0.183, 0.578])

    # get joint distributions
    joint_distribution = generate_distributions()

    number_of_samples = 1000000
    A, B, C, Y_A, Y_B, Y_C, S, ST = calculate_sensitivity_indices_non_additive_model(number_of_samples,
                                                                                     joint_distribution)

    print("First Order Indices           Total Sensitivity Indices\n")
    print('       S_est | S_ref                 ST_est | ST_ref\n')
    for k, (s, sb, s_t, sb_t) in enumerate(zip(S, S_book, ST, St_book)):
        print('S_{} : {:>6.3f} | {:2.3f}          ST_{} : {:>6.3f} | {:2.3f}'.format(k + 1, s, sb, k+1, s_t, sb_t))

# Scatter plots

In [5]:
    # Scatter plots of data for visual inspection of sensitivity
    scatter = plt.figure('Scatter plots')
    scatter.suptitle('Scatter plots')
    n_samples_to_plot = 500

    number_of_parameters = len(joint_distribution)
    n_factors = number_of_parameters/2
    for k in range(n_factors):
        plt.subplot(2, n_factors, k + 1)
        plt.plot(A[:n_samples_to_plot, k], Y_A[:n_samples_to_plot], '.')
        plt.xlabel('Z{}'.format(k+1))
        plt.ylabel('Y')
        plt.ylim([-150, 150])

        plt.subplot(2, n_factors, k + 1 + n_factors)
        plt.plot(A[:n_samples_to_plot, k+4], Y_A[:n_samples_to_plot], '.')
        plt.xlabel('W{}'.format(k+1))
        plt.ylabel('Y')
        plt.ylim([-150, 150])

# Convergence Analysis
This may take a while, be patient!
You can also reduce iteration number for averaging.

In [6]:
#     # Random sampling
#     list_of_samples = np.array([10000, 50000, 100000, 500000, 1000000])
#     s_err = np.zeros((len(list_of_samples), number_of_parameters))
#     st_err = np.zeros((len(list_of_samples), number_of_parameters))
#     # average over
#     number_of_iterations = 10
#     for i, n_samples in enumerate(list_of_samples):
#         for j in range(number_of_iterations):
#             A, B, C, Y_A, Y_B, Y_C, S, ST = calculate_sensitivity_indices_non_additive_model(n_samples,
#                                                                                              joint_distribution,
#                                                                                              sample_method='R')
#             s_err[i] += np.abs(S - S_book)
#             st_err[i] += np.abs(ST - St_book)
#         s_err[i] /= float(number_of_iterations)
#         st_err[i] /= float(number_of_iterations)
# 
#     fig_random = plt.figure('Random sampling - average of indices')
#     fig_random.suptitle('Random sampling - average of indices')
# 
#     ax = plt.subplot(1, 2, 1)
#     plt.title('First order sensitivity indices')
#     plt.plot(list_of_samples / 1000, np.sum(s_err, axis=1), '-')
#     ax.set_yscale('log')
#     plt.ylabel('abs error')
#     plt.xlabel('number of samples [1e3]')
# 
#     ax1 = plt.subplot(1, 2, 2)
#     plt.title('Total sensitivity indices')
#     plt.plot(list_of_samples / 1000, np.sum(st_err, axis=1), '-')
#     ax1.set_yscale('log')
#     plt.ylabel('abs error')
#     plt.xlabel('number of samples [1e3]')
#

Plotting the individual sensitivity indices:

In [7]:
#     # figure individual
#     fig_random = plt.figure('Random sampling')
#     fig_random.suptitle('Random sampling')
#     for l, (s_e, st_e) in enumerate(zip(s_err.T, st_err.T)):
#         ax = plt.subplot(1, 2, 1)
#         plt.title('First order sensitivity indices')
#         plt.plot(list_of_samples/1000, s_e, '-', label='S_{}'.format(l))
#         ax.set_yscale('log')
#         plt.ylabel('abs error')
#         plt.xlabel('number of samples [1e3]')
#         plt.legend()
# 
#         ax1 = plt.subplot(1, 2, 2)
#         plt.title('Total sensitivity indices')
#         plt.plot(list_of_samples/1000, st_e, '-', label='ST_{}'.format(l))
#         ax1.set_yscale('log')
#         plt.ylabel('abs error')
#         plt.xlabel('number of samples [1e3]')
#         plt.legend()
#

# References